In [19]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ML_Pipeline") \
    .getOrCreate()

df = spark.read.csv("../data/processed/data-encoded.csv", header=True, inferSchema=True)

### Gestion du déséquilibre de classes

In [20]:
major = df.filter(df.Exited == 0)
minor = df.filter(df.Exited == 1)

ratio = minor.count() / major.count()
major_sampled = major.sample(withReplacement=False, fraction=ratio)

df_balanced = major_sampled.union(minor)

### Sélection et assemblage des features

In [ ]:
from pyspark.ml.feature import VectorAssembler

input_cols = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 
              'Geography_vec', 'Gender_vec']

assembler = VectorAssembler(
    inputCols=input_cols,
    outputCol="features"
)

### Conversion de string en vecteur

In [29]:
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession

def str_to_vector(s):
    if s is None:
        return None
    try:
        values = [float(x) for x in s.strip("[]").split(",")]
        return Vectors.dense(values)
    except:
        return None

to_vector_udf = F.udf(str_to_vector, VectorUDT())

# Apply to both columns
df_fixed = (
    df_balanced.withColumn("Geography_vec", to_vector_udf("Geography_vec"))
      .withColumn("Gender_vec", to_vector_udf("Gender_vec"))
)

df_fixed.printSchema()

root
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)
 |-- Geography_index: double (nullable = true)
 |-- Gender_index: double (nullable = true)
 |-- Geography_vec: vector (nullable = true)
 |-- Gender_vec: vector (nullable = true)



### Sélection et assemblage des features

In [30]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(
    inputCol="features",
    outputCol="scaledFeatures",
    withMean=True,
    withStd=True
)

### Séparation du dataset

In [31]:
train_df, test_df = df_fixed.randomSplit([0.8, 0.2], seed=42)

### Choix du modèle MLlib

In [32]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    featuresCol="scaledFeatures",
    labelCol="Exited"
)

### Construction du Pipeline

In [33]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[
    assembler,
    scaler,
    lr
])

### Entraînement et évaluation

In [34]:
model = pipeline.fit(train_df)
predictions = model.transform(test_df)

predictions.select("Exited", "prediction", "probability").show(5)

Py4JJavaError: An error occurred while calling o169.transform.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 24.0 failed 1 times, most recent failure: Lost task 0.0 in stage 24.0 (TID 18) (Anass-PC executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:252)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:143)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:158)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:178)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:261)
	at org.apache.spark.sql.execution.python.BatchEvalPythonEvaluatorFactory.evaluate(BatchEvalPythonExec.scala:83)
	at org.apache.spark.sql.execution.python.EvalPythonEvaluatorFactory$EvalPythonPartitionEvaluator.eval(EvalPythonEvaluatorFactory.scala:113)
	

### Évaluation du modèle

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    labelCol="Exited",
    rawPredictionCol="prediction",
    metricName="areaUnderROC"
)

roc_auc = evaluator.evaluate(predictions)
print("AUC =", roc_auc)

### Sauvegarde du modèle

In [ ]:
model.write().overwrite().save("models/churn_model")